In [1]:
import os
import random
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F

SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
transform = transforms.ToTensor()

train = datasets.MNIST(root = '../data',train= True, transform= transform, download=True)
test = datasets.MNIST(root = '../data', train= False, transform= transform, download=True)

#train = datasets.MNIST(root = '../data',train= True, transform= transform)
#test = datasets.MNIST(root = '../data', train= False, transform= transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
print('number of training data: ', len(train))
print('number of test data: ', len(test))

number of training data:  60000
number of test data:  10000


In [5]:
indices = indices = list(range(len(train)))
np.random.shuffle(indices)

split = int(np.floor(0.9 * len(train)))
tr_idx, val_idx = indices[:split], indices[split:]


In [6]:
tr_sampler = SubsetRandomSampler(tr_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [7]:
train_loader = DataLoader(
    dataset=train,
    batch_size=10,
    num_workers=0,
    sampler=tr_sampler
)
val_loader = DataLoader(
    dataset=train,
    batch_size=10,
    num_workers=0,
    sampler=val_sampler
)
test_loader = DataLoader(
    dataset=test,
    batch_size=10,
    num_workers=0,
)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=0),
            torch.nn.ReLU())
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU())
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU())
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(288, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 288)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net()
model = model.to(device)

In [10]:
import torch.optim as optim

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [12]:
epochs = 10

for epoch in range(epochs):

    train_loss = 0
    train_total = 0
    train_correct = 0
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels) #loss 계산
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = outputs.max(1) #최종값으로 10개를 뽑게 되어있음 그것 중 가장 큰 값의 인덱스를 return해라
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item() #만족하면 1 만족못하면 0을 더함 (얼마나 correct한지)
    train_acc = train_correct / train_total #len(train)

    valid_loss = 0
    valid_total = 0
    valid_correct = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            valid_total += labels.size(0)
            valid_correct += (predicted == labels).sum().item()
    valid_acc = valid_correct / valid_total

    #print('Epoch: ', epoch + 1)
    #print('Loss: ', 'Train %.4f / Valid %.4f' % (train_loss, valid_loss))
    #print('Accuracy: ', 'Train %.4f / Valid %.4f' % (train_acc * 100, valid_acc * 100))

    print('[%d/%d] TrainLoss: %.3f, ValLoss: %.3f | TrainAcc: %.2f, ValAcc: %.2f' % 
            (epoch+1, epochs, train_loss, valid_loss, train_acc, valid_acc))

[1/10] TrainLoss: 3431.655, ValLoss: 87.948 | TrainAcc: 0.79, ValAcc: 0.96
[2/10] TrainLoss: 726.667, ValLoss: 70.184 | TrainAcc: 0.96, ValAcc: 0.96
[3/10] TrainLoss: 512.344, ValLoss: 48.713 | TrainAcc: 0.97, ValAcc: 0.97
[4/10] TrainLoss: 407.847, ValLoss: 37.336 | TrainAcc: 0.98, ValAcc: 0.98
[5/10] TrainLoss: 343.646, ValLoss: 37.126 | TrainAcc: 0.98, ValAcc: 0.98
[6/10] TrainLoss: 291.259, ValLoss: 34.315 | TrainAcc: 0.98, ValAcc: 0.98
[7/10] TrainLoss: 253.589, ValLoss: 27.250 | TrainAcc: 0.99, ValAcc: 0.98
[8/10] TrainLoss: 221.776, ValLoss: 26.733 | TrainAcc: 0.99, ValAcc: 0.98
[9/10] TrainLoss: 199.857, ValLoss: 26.230 | TrainAcc: 0.99, ValAcc: 0.99
[10/10] TrainLoss: 178.897, ValLoss: 26.214 | TrainAcc: 0.99, ValAcc: 0.99


In [13]:
torch.save(model.state_dict(), './2017310367_DasolJeong.pt')


In [14]:
with torch.no_grad():
    X_test = test.test_data.view(-1, 1, 28, 28).float().to(device)
    Y_test = test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('TestAcc: %.2f' % (accuracy.item()))

TestAcc: 0.99


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
